In [60]:
import requests
import lxml.html as lh
import pandas as pd
import scrapy

In [61]:
#example url
url='http://www.plusliga.pl/statsPlayers/id/304.html'

In [62]:
def scrap_table_from_url(url):
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)
    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)
    #Parse data that are stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath('//tr')
    
    return tr_elements

In [63]:
tr_elements = scrap_table_from_url(url)

In [64]:
#Check the length of all the rows
#we have two tables on the webpage
#two rows are shorter (7 elements), because both tables on webpage have two header lines
[len(T) for T in tr_elements[0:]]

[7,
 19,
 19,
 7,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19,
 19]

In [65]:
def create_headers(tr_elements):
    #create empty list
    col1=[]
    i=0
    
    #store headers from first row in table
    for t in tr_elements[0]:
        i+=1
        name=t.text_content()
        col1.append(name)
    
    #create empty list
    col2=[]
    j=0
    #store headers from second row in table
    for t in tr_elements[1]:
        j+=1
        name=t.text_content()
        col2.append(name)
    
    #create one header names for all table
    col2[3]=col1[3] + '_' + col2[3]
    col2[4]=col1[3] + '_' + col2[4]
    col2[5]=col1[3] + '_' + col2[5]
    col2[6]=col1[3] + '_' + col2[6]
    col2[7]=col1[4] + '_' + col2[7]
    col2[8]=col1[4] + '_' + col2[8]
    col2[9]=col1[4] + '_' + col2[9]
    col2[10]=col1[4] + '_' + col2[10]
    col2[11]=col1[4] + '_' + col2[11]
    col2[12]=col1[5] + '_' + col2[12]
    col2[13]=col1[5] + '_' + col2[13]
    col2[14]=col1[5] + '_' + col2[14]
    col2[15]=col1[5] + '_' + col2[15]
    col2[16]=col1[5] + '_' + col2[16]
    col2[17]=col1[6] + '_' + col2[17]
    col2[18]=col1[6] + '_' + col2[18]
    
    return col2

In [66]:
headers = create_headers(tr_elements)
headers

['Rozegrane mecze',
 'Sety',
 'Punkty',
 'Zagrywka_Suma',
 'Zagrywka_As',
 'Zagrywka_Błąd',
 'Zagrywka_Asy na set',
 'Przyjęcie_Suma',
 'Przyjęcie_Błąd',
 'Przyjęcie_Neg',
 'Przyjęcie_Perf',
 'Przyjęcie_% perf',
 'Atak_Suma',
 'Atak_Błąd',
 'Atak_Blok',
 'Atak_Perf',
 'Atak_% perf',
 'Blok_Pkt',
 'Blok_Pkt na set']

In [67]:
def make_space_in_headers(headers):
    #change list to tuple
    headers_tuple = tuple(headers)
    #create empty list
    col=[]
    #add empty lists to tuple
    for t in range(0,len(headers_tuple)):
        col.append((headers_tuple[t],[]))
    return col

In [68]:
col = make_space_in_headers(headers)

In [69]:
#Our data are stored on the second table, which is from 5th row onwards without last two rows
def fill_data(tr_elements,col):
    row_width = len(tr_elements[4])
    
    for j in range(5,len(tr_elements)-2):
        #T is our j'th row
        T=tr_elements[j]
    
        #If row is not of size row_width (19), the //tr data is not from our table 
        if len(T)!=row_width:
            break
    
        #i is the index of our column
        i=0

        #Iterate through each element of the row
        for t in T.iterchildren():
            data=t.text_content() 
            #Check if row is empty
            if i>0:
            #Convert any numerical value to integers
                try:
                    data=int(data)
                except:
                    pass
            #Append the data to the empty list of the i'th column
            col[i][1].append(data)
            #Increment i for the next column
            i+=1
    return col

In [70]:
col = fill_data(tr_elements,col)

In [71]:
#checking the length of each column
[len(C) for (title,C) in col]

[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]

In [72]:
#create dataframe
def create_dataframe(col):
    #creating dataframe from dict
    Dict={title:column for (title,column) in col}
    df=pd.DataFrame(Dict)
    
    return df

In [73]:
create_dataframe(col)

,Rozegrane mecze,Sety,Punkty,Zagrywka_Suma,Zagrywka_As,Zagrywka_Błąd,Zagrywka_Asy na set,Przyjęcie_Suma,Przyjęcie_Błąd,Przyjęcie_Neg,Przyjęcie_Perf,Przyjęcie_% perf,Atak_Suma,Atak_Błąd,Atak_Blok,Atak_Perf,Atak_% perf,Blok_Pkt,Blok_Pkt na set
0,Trefl Gdańsk -PGE Skra Bełchatów,4,22,20,1,3,"0,25",0,0,0,0,"0,00",39,2,3,20,"51,28",1,"0,25"
1,Trefl Gdańsk -Cerrad Enea Czarni Radom,4,19,17,3,0,"0,75",0,0,0,0,"0,00",28,0,3,15,"53,57",1,"0,25"
2,Trefl Gdańsk -GKS Katowice,5,17,16,1,5,"0,20",0,0,0,0,"0,00",38,4,3,15,"39,47",1,"0,20"
3,Trefl Gdańsk -Ślepsk Malow Suwałki,5,29,13,1,4,"0,20",0,0,0,0,"0,00",44,2,7,27,"61,36",1,"0,20"
4,Trefl Gdańsk -Indykpol AZS Olsztyn,4,23,20,2,6,"0,50",1,1,0,0,"0,00",41,2,4,19,"46,34",2,"0,50"
5,Trefl Gdańsk -Cuprum Lubin,3,18,10,1,1,"0,33",1,0,0,0,"0,00",27,2,1,12,"44,44",5,"1,67"
6,Trefl Gdańsk -BKS Visła Bydgoszcz,5,18,15,0,2,"0,00",2,1,1,0,"0,00",32,5,3,14,"43,75",4,"0,80"
7,Trefl Gdańsk -MKS Będzin,4,19,22,2,4,"0,50",1,0,1,0,"0,00",36,2,2,14,"38,89",3,"0,75"
8,Trefl Gdańsk -Grupa Azoty ZAKSA Kędzierzyn-Koźle,4,22,15,2,7,"0,50",1,0,1,0,"0,00",37,1,2,19,"51,35",1,"0,25"
9,Trefl Gdańsk -VERVA Warszawa ORLEN Paliwa,3,16,13,2,4,"0,67",0,0,0,0,"0,00",32,3,3,14,"43,75",0,"0,00"


In [74]:
#scrap name of the player 
def scrap_name(url):
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)
    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)
    #Parse data that are stored between <h1>..</h1> of HTML
    tr_elements = doc.xpath('//h1')
    name = tr_elements[0].text_content()
    
    return name

In [75]:
#scrap number of seasons played by this player in league 
def scrap_number_seasons(url):
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)
    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)
    #Parse data that are stored between <span>..</span> of HTML
    tr_elements = doc.xpath('//span')
    name = tr_elements[1267].text_content()
    
    return name

In [76]:
#scrap number of games played by this player in league 
def scrap_number_games(url):
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)
    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)
    #Parse data that are stored between <span>..</span> of HTML
    tr_elements = doc.xpath('//span')
    name = tr_elements[1268].text_content()
    
    return name

In [77]:
#scrap number of points scored by this player in league 
def scrap_number_points(url):
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)
    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)
    #Parse data that are stored between <span>..</span> of HTML
    tr_elements = doc.xpath('//span')
    name = tr_elements[1269].text_content()
    
    return name

In [78]:
#scrap number of aces served by this player in league 
def scrap_number_aces(url):
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)
    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)
    #Parse data that are stored between <span>..</span> of HTML
    tr_elements = doc.xpath('//span')
    name = tr_elements[1270].text_content()
    
    return name

In [79]:
#scrap number of blocks made by this player in league 
def scrap_number_blocks(url):
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)
    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)
    #Parse data that are stored between <span>..</span> of HTML
    tr_elements = doc.xpath('//span')
    name = tr_elements[1271].text_content()
    
    return name

In [80]:
#final function
def scrap_table_to_df_and_overall_stats(url):
    headers = create_headers(scrap_table_from_url(url))
    col = make_space_in_headers(headers)
    col1 = fill_data(scrap_table_from_url(url), col)
    df = create_dataframe(col1)
    df['Gracz'] = scrap_name(url)
    df['Liczba sezonów w lidze'] = scrap_number_seasons(url)
    df['Liczba meczy w lidze'] = scrap_number_games(url)
    df['Liczba pkt w lidze'] = scrap_number_points(url)
    df['Liczba asów w lidze'] = scrap_number_aces(url)
    df['Liczba bloków w lidze'] = scrap_number_blocks(url)
    return df

In [81]:
#test
achrem = 'http://www.plusliga.pl/statsPlayers/id/152.html'
df = scrap_table_to_df_and_overall_stats(achrem)

In [82]:
df

,Rozegrane mecze,Sety,Punkty,Zagrywka_Suma,Zagrywka_As,Zagrywka_Błąd,Zagrywka_Asy na set,Przyjęcie_Suma,Przyjęcie_Błąd,Przyjęcie_Neg,...,Atak_Perf,Atak_% perf,Blok_Pkt,Blok_Pkt na set,Gracz,Liczba sezonów w lidze,Liczba meczy w lidze,Liczba pkt w lidze,Liczba asów w lidze,Liczba bloków w lidze
0,Asseco Resovia Rzeszów -BKS Visła Bydgoszcz,3,1,4,1,0,"0,33",0,0,0,...,0,"0,00",0,"0,00",Alek Achrem,9,166,2007,152,181
1,Asseco Resovia Rzeszów -MKS Będzin,4,1,8,1,3,"0,25",0,0,0,...,0,"0,00",0,"0,00",Alek Achrem,9,166,2007,152,181
2,Asseco Resovia Rzeszów -Grupa Azoty ZAKSA Kędz...,2,0,2,0,2,"0,00",0,0,0,...,0,"0,00",0,"0,00",Alek Achrem,9,166,2007,152,181
3,Asseco Resovia Rzeszów -Cerrad Enea Czarni Radom,3,4,6,0,2,"0,00",13,2,3,...,4,"100,00",0,"0,00",Alek Achrem,9,166,2007,152,181


In [83]:
#list of all urls from 2019/2020 season
urls = [
    "http://www.plusliga.pl/statsPlayers/id/152.html",
    "http://www.plusliga.pl/statsPlayers/id/30339.html",
    "http://www.plusliga.pl/statsPlayers/id/118.html",
    "http://www.plusliga.pl/statsPlayers/id/30172.html",
    "http://www.plusliga.pl/statsPlayers/id/415.html",
    "http://www.plusliga.pl/statsPlayers/id/2100505.html",
    "http://www.plusliga.pl/statsPlayers/id/82.html",
    "http://www.plusliga.pl/statsPlayers/id/22767.html",
    "http://www.plusliga.pl/statsPlayers/id/277.html",
    "http://www.plusliga.pl/statsPlayers/id/631.html",
    "http://www.plusliga.pl/statsPlayers/id/297.html",
    "http://www.plusliga.pl/statsPlayers/id/293.html",
    "http://www.plusliga.pl/statsPlayers/id/30333.html",
    "http://www.plusliga.pl/statsPlayers/id/231.html",
    "http://www.plusliga.pl/statsPlayers/id/63.html",
    "http://www.plusliga.pl/statsPlayers/id/22550.html",
    "http://www.plusliga.pl/statsPlayers/id/101.html",
    "http://www.plusliga.pl/statsPlayers/id/488.html",
    "http://www.plusliga.pl/statsPlayers/id/2100506.html",
    "http://www.plusliga.pl/statsPlayers/id/30591.html",
    "http://www.plusliga.pl/statsPlayers/id/2100349.html",
    "http://www.plusliga.pl/statsPlayers/id/36.html",
    "http://www.plusliga.pl/statsPlayers/id/563.html",
    "http://www.plusliga.pl/statsPlayers/id/29750.html",
    "http://www.plusliga.pl/statsPlayers/id/30510.html",
    "http://www.plusliga.pl/statsPlayers/id/484.html",
    "http://www.plusliga.pl/statsPlayers/id/473.html",
    "http://www.plusliga.pl/statsPlayers/id/30003.html",
    "http://www.plusliga.pl/statsPlayers/id/2100361.html",
    "http://www.plusliga.pl/statsPlayers/id/30331.html",
    "http://www.plusliga.pl/statsPlayers/id/22529.html",
    "http://www.plusliga.pl/statsPlayers/id/2100364.html",
    "http://www.plusliga.pl/statsPlayers/id/377.html",
    "http://www.plusliga.pl/statsPlayers/id/30579.html",
    "http://www.plusliga.pl/statsPlayers/id/22706.html",
    "http://www.plusliga.pl/statsPlayers/id/474.html",
    "http://www.plusliga.pl/statsPlayers/id/30476.html",
    "http://www.plusliga.pl/statsPlayers/id/304.html",
    "http://www.plusliga.pl/statsPlayers/id/22534.html",
    "http://www.plusliga.pl/statsPlayers/id/22856.html",
    "http://www.plusliga.pl/statsPlayers/id/26899.html",
    "http://www.plusliga.pl/statsPlayers/id/632.html",
    "http://www.plusliga.pl/statsPlayers/id/22806.html",
    "http://www.plusliga.pl/statsPlayers/id/30495.html",
    "http://www.plusliga.pl/statsPlayers/id/99.html",
    "http://www.plusliga.pl/statsPlayers/id/22.html",
    "http://www.plusliga.pl/statsPlayers/id/22808.html",
    "http://www.plusliga.pl/statsPlayers/id/2100356.html",
    "http://www.plusliga.pl/statsPlayers/id/105.html",
    "http://www.plusliga.pl/statsPlayers/id/29832.html",
    "http://www.plusliga.pl/statsPlayers/id/459.html",
    "http://www.plusliga.pl/statsPlayers/id/470.html",
    "http://www.plusliga.pl/statsPlayers/id/26825.html",
    "http://www.plusliga.pl/statsPlayers/id/2100507.html",
    "http://www.plusliga.pl/statsPlayers/id/25754.html",
    "http://www.plusliga.pl/statsPlayers/id/22800.html",
    "http://www.plusliga.pl/statsPlayers/id/2100257.html",
    "http://www.plusliga.pl/statsPlayers/id/22531.html",
    "http://www.plusliga.pl/statsPlayers/id/113.html",
    "http://www.plusliga.pl/statsPlayers/id/78.html",
    "http://www.plusliga.pl/statsPlayers/id/29821.html",
    "http://www.plusliga.pl/statsPlayers/id/2100363.html",
    "http://www.plusliga.pl/statsPlayers/id/380.html",
    "http://www.plusliga.pl/statsPlayers/id/22790.html",
    "http://www.plusliga.pl/statsPlayers/id/30489.html",
    "http://www.plusliga.pl/statsPlayers/id/29756.html",
    "http://www.plusliga.pl/statsPlayers/id/2100522.html",
    "http://www.plusliga.pl/statsPlayers/id/28000.html",
    "http://www.plusliga.pl/statsPlayers/id/26870.html",
    "http://www.plusliga.pl/statsPlayers/id/30496.html",
    "http://www.plusliga.pl/statsPlayers/id/26900.html",
    "http://www.plusliga.pl/statsPlayers/id/2100528.html",
    "http://www.plusliga.pl/statsPlayers/id/272.html",
    "http://www.plusliga.pl/statsPlayers/id/6.html",
    "http://www.plusliga.pl/statsPlayers/id/358.html",
    "http://www.plusliga.pl/statsPlayers/id/28378.html",
    "http://www.plusliga.pl/statsPlayers/id/22708.html",
    "http://www.plusliga.pl/statsPlayers/id/26823.html",
    "http://www.plusliga.pl/statsPlayers/id/573.html",
    "http://www.plusliga.pl/statsPlayers/id/2100359.html",
    "http://www.plusliga.pl/statsPlayers/id/29811.html",
    "http://www.plusliga.pl/statsPlayers/id/477.html",
    "http://www.plusliga.pl/statsPlayers/id/29719.html",
    "http://www.plusliga.pl/statsPlayers/id/103.html",
    "http://www.plusliga.pl/statsPlayers/id/707.html",
    "http://www.plusliga.pl/statsPlayers/id/30353.html",
    "http://www.plusliga.pl/statsPlayers/id/29886.html",
    "http://www.plusliga.pl/statsPlayers/id/561.html",
    "http://www.plusliga.pl/statsPlayers/id/88.html",
    "http://www.plusliga.pl/statsPlayers/id/2100513.html",
    "http://www.plusliga.pl/statsPlayers/id/58.html",
    "http://www.plusliga.pl/statsPlayers/id/2100355.html",
    "http://www.plusliga.pl/statsPlayers/id/28395.html",
    "http://www.plusliga.pl/statsPlayers/id/235.html",
    "http://www.plusliga.pl/statsPlayers/id/370.html",
    "http://www.plusliga.pl/statsPlayers/id/2100360.html",
    "http://www.plusliga.pl/statsPlayers/id/22640.html",
    "http://www.plusliga.pl/statsPlayers/id/2100525.html",
    "http://www.plusliga.pl/statsPlayers/id/177.html",
    "http://www.plusliga.pl/statsPlayers/id/516.html",
    "http://www.plusliga.pl/statsPlayers/id/22709.html",
    "http://www.plusliga.pl/statsPlayers/id/2100357.html",
    "http://www.plusliga.pl/statsPlayers/id/22573.html",
    "http://www.plusliga.pl/statsPlayers/id/22782.html",
    "http://www.plusliga.pl/statsPlayers/id/22799.html",
    "http://www.plusliga.pl/statsPlayers/id/87.html",
    "http://www.plusliga.pl/statsPlayers/id/28390.html",
    "http://www.plusliga.pl/statsPlayers/id/32.html",
    "http://www.plusliga.pl/statsPlayers/id/289.html",
    "http://www.plusliga.pl/statsPlayers/id/29662.html",
    "http://www.plusliga.pl/statsPlayers/id/22796.html",
    "http://www.plusliga.pl/statsPlayers/id/26880.html",
    "http://www.plusliga.pl/statsPlayers/id/430.html",
    "http://www.plusliga.pl/statsPlayers/id/22572.html",
    "http://www.plusliga.pl/statsPlayers/id/424.html",
    "http://www.plusliga.pl/statsPlayers/id/164.html",
    "http://www.plusliga.pl/statsPlayers/id/2100034.html",
    "http://www.plusliga.pl/statsPlayers/id/22526.html",
    "http://www.plusliga.pl/statsPlayers/id/45.html",
    "http://www.plusliga.pl/statsPlayers/id/153.html",
    "http://www.plusliga.pl/statsPlayers/id/107.html",
    "http://www.plusliga.pl/statsPlayers/id/570.html",
    "http://www.plusliga.pl/statsPlayers/id/30329.html",
    "http://www.plusliga.pl/statsPlayers/id/2100500.html",
    "http://www.plusliga.pl/statsPlayers/id/83.html",
    "http://www.plusliga.pl/statsPlayers/id/2100001.html",
    "http://www.plusliga.pl/statsPlayers/id/22804.html",
    "http://www.plusliga.pl/statsPlayers/id/414.html",
    "http://www.plusliga.pl/statsPlayers/id/71.html",
    "http://www.plusliga.pl/statsPlayers/id/467.html",
    "http://www.plusliga.pl/statsPlayers/id/709.html",
    "http://www.plusliga.pl/statsPlayers/id/22528.html",
    "http://www.plusliga.pl/statsPlayers/id/382.html",
    "http://www.plusliga.pl/statsPlayers/id/22569.html",
    "http://www.plusliga.pl/statsPlayers/id/2100502.html",
    "http://www.plusliga.pl/statsPlayers/id/2100521.html",
    "http://www.plusliga.pl/statsPlayers/id/670.html",
    "http://www.plusliga.pl/statsPlayers/id/30508.html",
    "http://www.plusliga.pl/statsPlayers/id/162.html",
    "http://www.plusliga.pl/statsPlayers/id/2100354.html",
    "http://www.plusliga.pl/statsPlayers/id/282.html",
    "http://www.plusliga.pl/statsPlayers/id/29747.html",
    "http://www.plusliga.pl/statsPlayers/id/2100249.html",
    "http://www.plusliga.pl/statsPlayers/id/589.html",
    "http://www.plusliga.pl/statsPlayers/id/30498.html",
    "http://www.plusliga.pl/statsPlayers/id/656.html",
    "http://www.plusliga.pl/statsPlayers/id/2100367.html",
    "http://www.plusliga.pl/statsPlayers/id/2100353.html",
    "http://www.plusliga.pl/statsPlayers/id/30342.html",
    "http://www.plusliga.pl/statsPlayers/id/450.html",
    "http://www.plusliga.pl/statsPlayers/id/483.html",
    "http://www.plusliga.pl/statsPlayers/id/298.html",
    "http://www.plusliga.pl/statsPlayers/id/30341.html",
    "http://www.plusliga.pl/statsPlayers/id/51.html",
    "http://www.plusliga.pl/statsPlayers/id/2100155.html",
    "http://www.plusliga.pl/statsPlayers/id/35.html",
    "http://www.plusliga.pl/statsPlayers/id/22560.html",
    "http://www.plusliga.pl/statsPlayers/id/2100518.html",
    "http://www.plusliga.pl/statsPlayers/id/29785.html",
    "http://www.plusliga.pl/statsPlayers/id/2100157.html",
    "http://www.plusliga.pl/statsPlayers/id/30482.html",
    "http://www.plusliga.pl/statsPlayers/id/2100115.html",
    "http://www.plusliga.pl/statsPlayers/id/30484.html",
    "http://www.plusliga.pl/statsPlayers/id/704.html",
    "http://www.plusliga.pl/statsPlayers/id/27949.html",
    "http://www.plusliga.pl/statsPlayers/id/2100366.html",
    "http://www.plusliga.pl/statsPlayers/id/30506.html",
    "http://www.plusliga.pl/statsPlayers/id/624.html",
    "http://www.plusliga.pl/statsPlayers/id/2100531.html",
    "http://www.plusliga.pl/statsPlayers/id/2100151.html",
    "http://www.plusliga.pl/statsPlayers/id/29833.html",
    "http://www.plusliga.pl/statsPlayers/id/2100193.html",
    "http://www.plusliga.pl/statsPlayers/id/327.html",
    "http://www.plusliga.pl/statsPlayers/id/456.html",
    "http://www.plusliga.pl/statsPlayers/id/29969.html",
    "http://www.plusliga.pl/statsPlayers/id/2100497.html",
    "http://www.plusliga.pl/statsPlayers/id/68.html",
    "http://www.plusliga.pl/statsPlayers/id/2100503.html",
    "http://www.plusliga.pl/statsPlayers/id/649.html",
    "http://www.plusliga.pl/statsPlayers/id/22758.html",
    "http://www.plusliga.pl/statsPlayers/id/30297.html",
    "http://www.plusliga.pl/statsPlayers/id/252.html",
    "http://www.plusliga.pl/statsPlayers/id/22558.html",
    "http://www.plusliga.pl/statsPlayers/id/413.html",
    "http://www.plusliga.pl/statsPlayers/id/30505.html",
    "http://www.plusliga.pl/statsPlayers/id/2100147.html",
    "http://www.plusliga.pl/statsPlayers/id/29753.html",
    "http://www.plusliga.pl/statsPlayers/id/28581.html",
    "http://www.plusliga.pl/statsPlayers/id/633.html",
    "http://www.plusliga.pl/statsPlayers/id/2100358.html",
    "http://www.plusliga.pl/statsPlayers/id/630.html",
    "http://www.plusliga.pl/statsPlayers/id/27987.html",
    "http://www.plusliga.pl/statsPlayers/id/2100362.html",
    "http://www.plusliga.pl/statsPlayers/id/28372.html",
    "http://www.plusliga.pl/statsPlayers/id/28371.html",
    "http://www.plusliga.pl/statsPlayers/id/2100369.html",
    "http://www.plusliga.pl/statsPlayers/id/28110.html",
    "http://www.plusliga.pl/statsPlayers/id/291.html",
    "http://www.plusliga.pl/statsPlayers/id/2100348.html",
    "http://www.plusliga.pl/statsPlayers/id/2100351.html",
    "http://www.plusliga.pl/statsPlayers/id/30493.html",
    "http://www.plusliga.pl/statsPlayers/id/667.html",
    "http://www.plusliga.pl/statsPlayers/id/26868.html",
    "http://www.plusliga.pl/statsPlayers/id/29606.html",
    "http://www.plusliga.pl/statsPlayers/id/156.html",
    "http://www.plusliga.pl/statsPlayers/id/444.html",
    "http://www.plusliga.pl/statsPlayers/id/29834.html",
    "http://www.plusliga.pl/statsPlayers/id/2100512.html",
    "http://www.plusliga.pl/statsPlayers/id/2100365.html",
    "http://www.plusliga.pl/statsPlayers/id/2100368.html",
    "http://www.plusliga.pl/statsPlayers/id/73.html",
    "http://www.plusliga.pl/statsPlayers/id/3.html",
    "http://www.plusliga.pl/statsPlayers/id/179.html",
    "http://www.plusliga.pl/statsPlayers/id/29752.html",
    "http://www.plusliga.pl/statsPlayers/id/28.html",
    "http://www.plusliga.pl/statsPlayers/id/112.html",
    "http://www.plusliga.pl/statsPlayers/id/80.html",
    "http://www.plusliga.pl/statsPlayers/id/22802.html",
    "http://www.plusliga.pl/statsPlayers/id/81.html",
    "http://www.plusliga.pl/statsPlayers/id/30500.html",
    "http://www.plusliga.pl/statsPlayers/id/22805.html",
    "http://www.plusliga.pl/statsPlayers/id/372.html",
    "http://www.plusliga.pl/statsPlayers/id/620.html",
    "http://www.plusliga.pl/statsPlayers/id/60.html",
    "http://www.plusliga.pl/statsPlayers/id/442.html",
    "http://www.plusliga.pl/statsPlayers/id/152.html",
    "http://www.plusliga.pl/statsPlayers/id/30339.html",
    "http://www.plusliga.pl/statsPlayers/id/118.html",
    "http://www.plusliga.pl/statsPlayers/id/30172.html",
    "http://www.plusliga.pl/statsPlayers/id/415.html",
    "http://www.plusliga.pl/statsPlayers/id/2100505.html",
    "http://www.plusliga.pl/statsPlayers/id/82.html",
    "http://www.plusliga.pl/statsPlayers/id/22767.html",
    "http://www.plusliga.pl/statsPlayers/id/277.html",
    "http://www.plusliga.pl/statsPlayers/id/631.html",
    "http://www.plusliga.pl/statsPlayers/id/297.html",
    "http://www.plusliga.pl/statsPlayers/id/293.html",
    "http://www.plusliga.pl/statsPlayers/id/30333.html",
    "http://www.plusliga.pl/statsPlayers/id/231.html",
    "http://www.plusliga.pl/statsPlayers/id/63.html",
    "http://www.plusliga.pl/statsPlayers/id/22550.html",
    "http://www.plusliga.pl/statsPlayers/id/101.html",
    "http://www.plusliga.pl/statsPlayers/id/488.html",
    "http://www.plusliga.pl/statsPlayers/id/2100506.html",
    "http://www.plusliga.pl/statsPlayers/id/30591.html",
    "http://www.plusliga.pl/statsPlayers/id/2100349.html",
    "http://www.plusliga.pl/statsPlayers/id/36.html",
    "http://www.plusliga.pl/statsPlayers/id/563.html",
    "http://www.plusliga.pl/statsPlayers/id/29750.html",
    "http://www.plusliga.pl/statsPlayers/id/30510.html",
    "http://www.plusliga.pl/statsPlayers/id/484.html",
    "http://www.plusliga.pl/statsPlayers/id/473.html",
    "http://www.plusliga.pl/statsPlayers/id/30003.html",
    "http://www.plusliga.pl/statsPlayers/id/2100361.html",
    "http://www.plusliga.pl/statsPlayers/id/30331.html",
    "http://www.plusliga.pl/statsPlayers/id/22529.html",
    "http://www.plusliga.pl/statsPlayers/id/2100364.html",
    "http://www.plusliga.pl/statsPlayers/id/377.html",
    "http://www.plusliga.pl/statsPlayers/id/30579.html",
    "http://www.plusliga.pl/statsPlayers/id/22706.html",
    "http://www.plusliga.pl/statsPlayers/id/474.html",
    "http://www.plusliga.pl/statsPlayers/id/30476.html",
    "http://www.plusliga.pl/statsPlayers/id/304.html",
    "http://www.plusliga.pl/statsPlayers/id/22534.html",
    "http://www.plusliga.pl/statsPlayers/id/22856.html",
    "http://www.plusliga.pl/statsPlayers/id/26899.html",
    "http://www.plusliga.pl/statsPlayers/id/632.html",
    "http://www.plusliga.pl/statsPlayers/id/22806.html",
    "http://www.plusliga.pl/statsPlayers/id/30495.html",
    "http://www.plusliga.pl/statsPlayers/id/99.html",
    "http://www.plusliga.pl/statsPlayers/id/22.html",
    "http://www.plusliga.pl/statsPlayers/id/22808.html",
    "http://www.plusliga.pl/statsPlayers/id/2100356.html",
    "http://www.plusliga.pl/statsPlayers/id/105.html",
    "http://www.plusliga.pl/statsPlayers/id/29832.html",
    "http://www.plusliga.pl/statsPlayers/id/459.html",
    "http://www.plusliga.pl/statsPlayers/id/470.html",
    "http://www.plusliga.pl/statsPlayers/id/26825.html",
    "http://www.plusliga.pl/statsPlayers/id/2100507.html",
    "http://www.plusliga.pl/statsPlayers/id/25754.html",
    "http://www.plusliga.pl/statsPlayers/id/22800.html",
    "http://www.plusliga.pl/statsPlayers/id/2100257.html",
    "http://www.plusliga.pl/statsPlayers/id/22531.html",
    "http://www.plusliga.pl/statsPlayers/id/113.html",
    "http://www.plusliga.pl/statsPlayers/id/78.html",
    "http://www.plusliga.pl/statsPlayers/id/29821.html",
    "http://www.plusliga.pl/statsPlayers/id/2100363.html",
    "http://www.plusliga.pl/statsPlayers/id/380.html",
    "http://www.plusliga.pl/statsPlayers/id/22790.html",
    "http://www.plusliga.pl/statsPlayers/id/30489.html",
    "http://www.plusliga.pl/statsPlayers/id/29756.html",
    "http://www.plusliga.pl/statsPlayers/id/2100522.html",
    "http://www.plusliga.pl/statsPlayers/id/28000.html",
    "http://www.plusliga.pl/statsPlayers/id/26870.html",
    "http://www.plusliga.pl/statsPlayers/id/30496.html",
    "http://www.plusliga.pl/statsPlayers/id/26900.html",
    "http://www.plusliga.pl/statsPlayers/id/2100528.html",
    "http://www.plusliga.pl/statsPlayers/id/272.html",
    "http://www.plusliga.pl/statsPlayers/id/6.html",
    "http://www.plusliga.pl/statsPlayers/id/358.html",
    "http://www.plusliga.pl/statsPlayers/id/28378.html",
    "http://www.plusliga.pl/statsPlayers/id/22708.html",
    "http://www.plusliga.pl/statsPlayers/id/26823.html",
    "http://www.plusliga.pl/statsPlayers/id/573.html",
    "http://www.plusliga.pl/statsPlayers/id/2100359.html",
    "http://www.plusliga.pl/statsPlayers/id/29811.html",
    "http://www.plusliga.pl/statsPlayers/id/477.html",
    "http://www.plusliga.pl/statsPlayers/id/29719.html",
    "http://www.plusliga.pl/statsPlayers/id/103.html",
    "http://www.plusliga.pl/statsPlayers/id/707.html",
    "http://www.plusliga.pl/statsPlayers/id/30353.html",
    "http://www.plusliga.pl/statsPlayers/id/29886.html",
    "http://www.plusliga.pl/statsPlayers/id/561.html",
    "http://www.plusliga.pl/statsPlayers/id/88.html",
    "http://www.plusliga.pl/statsPlayers/id/2100513.html",
    "http://www.plusliga.pl/statsPlayers/id/58.html",
    "http://www.plusliga.pl/statsPlayers/id/2100355.html",
    "http://www.plusliga.pl/statsPlayers/id/28395.html",
    "http://www.plusliga.pl/statsPlayers/id/235.html",
    "http://www.plusliga.pl/statsPlayers/id/370.html",
    "http://www.plusliga.pl/statsPlayers/id/2100360.html",
    "http://www.plusliga.pl/statsPlayers/id/22640.html",
    "http://www.plusliga.pl/statsPlayers/id/2100525.html",
    "http://www.plusliga.pl/statsPlayers/id/177.html",
    "http://www.plusliga.pl/statsPlayers/id/516.html",
    "http://www.plusliga.pl/statsPlayers/id/22709.html",
    "http://www.plusliga.pl/statsPlayers/id/2100357.html",
    "http://www.plusliga.pl/statsPlayers/id/22573.html",
    "http://www.plusliga.pl/statsPlayers/id/22782.html",
    "http://www.plusliga.pl/statsPlayers/id/22799.html",
    "http://www.plusliga.pl/statsPlayers/id/87.html",
    "http://www.plusliga.pl/statsPlayers/id/28390.html",
    "http://www.plusliga.pl/statsPlayers/id/32.html",
    "http://www.plusliga.pl/statsPlayers/id/289.html",
    "http://www.plusliga.pl/statsPlayers/id/29662.html",
    "http://www.plusliga.pl/statsPlayers/id/22796.html",
    "http://www.plusliga.pl/statsPlayers/id/26880.html",
    "http://www.plusliga.pl/statsPlayers/id/430.html",
    "http://www.plusliga.pl/statsPlayers/id/22572.html",
    "http://www.plusliga.pl/statsPlayers/id/424.html",
    "http://www.plusliga.pl/statsPlayers/id/164.html",
    "http://www.plusliga.pl/statsPlayers/id/2100034.html",
    "http://www.plusliga.pl/statsPlayers/id/22526.html",
    "http://www.plusliga.pl/statsPlayers/id/45.html",
    "http://www.plusliga.pl/statsPlayers/id/153.html",
    "http://www.plusliga.pl/statsPlayers/id/107.html",
    "http://www.plusliga.pl/statsPlayers/id/570.html",
    "http://www.plusliga.pl/statsPlayers/id/30329.html",
    "http://www.plusliga.pl/statsPlayers/id/2100500.html",
    "http://www.plusliga.pl/statsPlayers/id/83.html",
    "http://www.plusliga.pl/statsPlayers/id/2100001.html",
    "http://www.plusliga.pl/statsPlayers/id/22804.html",
    "http://www.plusliga.pl/statsPlayers/id/414.html",
    "http://www.plusliga.pl/statsPlayers/id/71.html",
    "http://www.plusliga.pl/statsPlayers/id/467.html",
    "http://www.plusliga.pl/statsPlayers/id/709.html",
    "http://www.plusliga.pl/statsPlayers/id/22528.html",
    "http://www.plusliga.pl/statsPlayers/id/382.html",
    "http://www.plusliga.pl/statsPlayers/id/22569.html",
    "http://www.plusliga.pl/statsPlayers/id/2100502.html",
    "http://www.plusliga.pl/statsPlayers/id/2100521.html",
    "http://www.plusliga.pl/statsPlayers/id/670.html",
    "http://www.plusliga.pl/statsPlayers/id/30508.html",
    "http://www.plusliga.pl/statsPlayers/id/162.html",
    "http://www.plusliga.pl/statsPlayers/id/2100354.html",
    "http://www.plusliga.pl/statsPlayers/id/282.html",
    "http://www.plusliga.pl/statsPlayers/id/29747.html",
    "http://www.plusliga.pl/statsPlayers/id/2100249.html",
    "http://www.plusliga.pl/statsPlayers/id/589.html",
    "http://www.plusliga.pl/statsPlayers/id/30498.html",
    "http://www.plusliga.pl/statsPlayers/id/656.html",
    "http://www.plusliga.pl/statsPlayers/id/2100367.html",
    "http://www.plusliga.pl/statsPlayers/id/2100353.html",
    "http://www.plusliga.pl/statsPlayers/id/30342.html",
    "http://www.plusliga.pl/statsPlayers/id/450.html",
    "http://www.plusliga.pl/statsPlayers/id/483.html",
    "http://www.plusliga.pl/statsPlayers/id/298.html",
    "http://www.plusliga.pl/statsPlayers/id/30341.html",
    "http://www.plusliga.pl/statsPlayers/id/51.html",
    "http://www.plusliga.pl/statsPlayers/id/2100155.html",
    "http://www.plusliga.pl/statsPlayers/id/35.html",
    "http://www.plusliga.pl/statsPlayers/id/22560.html",
    "http://www.plusliga.pl/statsPlayers/id/2100518.html",
    "http://www.plusliga.pl/statsPlayers/id/29785.html",
    "http://www.plusliga.pl/statsPlayers/id/2100157.html",
    "http://www.plusliga.pl/statsPlayers/id/30482.html",
    "http://www.plusliga.pl/statsPlayers/id/2100115.html",
    "http://www.plusliga.pl/statsPlayers/id/30484.html",
    "http://www.plusliga.pl/statsPlayers/id/704.html",
    "http://www.plusliga.pl/statsPlayers/id/27949.html",
    "http://www.plusliga.pl/statsPlayers/id/2100366.html",
    "http://www.plusliga.pl/statsPlayers/id/30506.html",
    "http://www.plusliga.pl/statsPlayers/id/624.html",
    "http://www.plusliga.pl/statsPlayers/id/2100531.html",
    "http://www.plusliga.pl/statsPlayers/id/2100151.html",
    "http://www.plusliga.pl/statsPlayers/id/29833.html",
    "http://www.plusliga.pl/statsPlayers/id/2100193.html",
    "http://www.plusliga.pl/statsPlayers/id/327.html",
    "http://www.plusliga.pl/statsPlayers/id/456.html",
    "http://www.plusliga.pl/statsPlayers/id/29969.html",
    "http://www.plusliga.pl/statsPlayers/id/2100497.html",
    "http://www.plusliga.pl/statsPlayers/id/68.html",
    "http://www.plusliga.pl/statsPlayers/id/2100503.html",
    "http://www.plusliga.pl/statsPlayers/id/649.html",
    "http://www.plusliga.pl/statsPlayers/id/22758.html",
    "http://www.plusliga.pl/statsPlayers/id/30297.html",
    "http://www.plusliga.pl/statsPlayers/id/252.html",
    "http://www.plusliga.pl/statsPlayers/id/22558.html",
    "http://www.plusliga.pl/statsPlayers/id/413.html",
    "http://www.plusliga.pl/statsPlayers/id/30505.html",
    "http://www.plusliga.pl/statsPlayers/id/2100147.html",
    "http://www.plusliga.pl/statsPlayers/id/29753.html",
    "http://www.plusliga.pl/statsPlayers/id/28581.html",
    "http://www.plusliga.pl/statsPlayers/id/633.html",
    "http://www.plusliga.pl/statsPlayers/id/2100358.html",
    "http://www.plusliga.pl/statsPlayers/id/630.html",
    "http://www.plusliga.pl/statsPlayers/id/27987.html",
    "http://www.plusliga.pl/statsPlayers/id/2100362.html",
    "http://www.plusliga.pl/statsPlayers/id/28372.html",
    "http://www.plusliga.pl/statsPlayers/id/28371.html",
    "http://www.plusliga.pl/statsPlayers/id/2100369.html",
    "http://www.plusliga.pl/statsPlayers/id/28110.html",
    "http://www.plusliga.pl/statsPlayers/id/291.html",
    "http://www.plusliga.pl/statsPlayers/id/2100348.html",
    "http://www.plusliga.pl/statsPlayers/id/2100351.html",
    "http://www.plusliga.pl/statsPlayers/id/30493.html",
    "http://www.plusliga.pl/statsPlayers/id/667.html",
    "http://www.plusliga.pl/statsPlayers/id/26868.html",
    "http://www.plusliga.pl/statsPlayers/id/29606.html",
    "http://www.plusliga.pl/statsPlayers/id/156.html",
    "http://www.plusliga.pl/statsPlayers/id/444.html",
    "http://www.plusliga.pl/statsPlayers/id/29834.html",
    "http://www.plusliga.pl/statsPlayers/id/2100512.html",
    "http://www.plusliga.pl/statsPlayers/id/2100365.html",
    "http://www.plusliga.pl/statsPlayers/id/2100368.html",
    "http://www.plusliga.pl/statsPlayers/id/73.html",
    "http://www.plusliga.pl/statsPlayers/id/3.html",
    "http://www.plusliga.pl/statsPlayers/id/179.html",
    "http://www.plusliga.pl/statsPlayers/id/29752.html",
    "http://www.plusliga.pl/statsPlayers/id/28.html",
    "http://www.plusliga.pl/statsPlayers/id/112.html",
    "http://www.plusliga.pl/statsPlayers/id/80.html",
    "http://www.plusliga.pl/statsPlayers/id/22802.html",
    "http://www.plusliga.pl/statsPlayers/id/81.html",
    "http://www.plusliga.pl/statsPlayers/id/30500.html",
    "http://www.plusliga.pl/statsPlayers/id/22805.html",
    "http://www.plusliga.pl/statsPlayers/id/372.html",
    "http://www.plusliga.pl/statsPlayers/id/620.html",
    "http://www.plusliga.pl/statsPlayers/id/60.html",
    "http://www.plusliga.pl/statsPlayers/id/442.html",
    "http://www.plusliga.pl/statsPlayers/id/152.html",
    "http://www.plusliga.pl/statsPlayers/id/30339.html",
    "http://www.plusliga.pl/statsPlayers/id/118.html",
    "http://www.plusliga.pl/statsPlayers/id/30172.html",
    "http://www.plusliga.pl/statsPlayers/id/415.html",
    "http://www.plusliga.pl/statsPlayers/id/2100505.html",
    "http://www.plusliga.pl/statsPlayers/id/82.html",
    "http://www.plusliga.pl/statsPlayers/id/22767.html",
    "http://www.plusliga.pl/statsPlayers/id/277.html",
    "http://www.plusliga.pl/statsPlayers/id/631.html",
    "http://www.plusliga.pl/statsPlayers/id/297.html",
    "http://www.plusliga.pl/statsPlayers/id/293.html",
    "http://www.plusliga.pl/statsPlayers/id/30333.html",
    "http://www.plusliga.pl/statsPlayers/id/231.html",
    "http://www.plusliga.pl/statsPlayers/id/63.html",
    "http://www.plusliga.pl/statsPlayers/id/22550.html",
    "http://www.plusliga.pl/statsPlayers/id/101.html",
    "http://www.plusliga.pl/statsPlayers/id/488.html",
    "http://www.plusliga.pl/statsPlayers/id/2100506.html",
    "http://www.plusliga.pl/statsPlayers/id/30591.html",
    "http://www.plusliga.pl/statsPlayers/id/2100349.html",
    "http://www.plusliga.pl/statsPlayers/id/36.html",
    "http://www.plusliga.pl/statsPlayers/id/563.html",
    "http://www.plusliga.pl/statsPlayers/id/29750.html",
    "http://www.plusliga.pl/statsPlayers/id/30510.html",
    "http://www.plusliga.pl/statsPlayers/id/484.html",
    "http://www.plusliga.pl/statsPlayers/id/473.html",
    "http://www.plusliga.pl/statsPlayers/id/30003.html",
    "http://www.plusliga.pl/statsPlayers/id/2100361.html",
    "http://www.plusliga.pl/statsPlayers/id/30331.html",
    "http://www.plusliga.pl/statsPlayers/id/22529.html",
    "http://www.plusliga.pl/statsPlayers/id/2100364.html",
    "http://www.plusliga.pl/statsPlayers/id/377.html",
    "http://www.plusliga.pl/statsPlayers/id/30579.html",
    "http://www.plusliga.pl/statsPlayers/id/22706.html",
    "http://www.plusliga.pl/statsPlayers/id/474.html",
    "http://www.plusliga.pl/statsPlayers/id/30476.html",
    "http://www.plusliga.pl/statsPlayers/id/304.html",
    "http://www.plusliga.pl/statsPlayers/id/22534.html",
    "http://www.plusliga.pl/statsPlayers/id/22856.html",
    "http://www.plusliga.pl/statsPlayers/id/26899.html",
    "http://www.plusliga.pl/statsPlayers/id/632.html",
    "http://www.plusliga.pl/statsPlayers/id/22806.html",
    "http://www.plusliga.pl/statsPlayers/id/30495.html",
    "http://www.plusliga.pl/statsPlayers/id/99.html",
    "http://www.plusliga.pl/statsPlayers/id/22.html",
    "http://www.plusliga.pl/statsPlayers/id/22808.html",
    "http://www.plusliga.pl/statsPlayers/id/2100356.html",
    "http://www.plusliga.pl/statsPlayers/id/105.html",
    "http://www.plusliga.pl/statsPlayers/id/29832.html",
    "http://www.plusliga.pl/statsPlayers/id/459.html",
    "http://www.plusliga.pl/statsPlayers/id/470.html",
    "http://www.plusliga.pl/statsPlayers/id/26825.html",
    "http://www.plusliga.pl/statsPlayers/id/2100507.html",
    "http://www.plusliga.pl/statsPlayers/id/25754.html",
    "http://www.plusliga.pl/statsPlayers/id/22800.html",
    "http://www.plusliga.pl/statsPlayers/id/2100257.html",
    "http://www.plusliga.pl/statsPlayers/id/22531.html",
    "http://www.plusliga.pl/statsPlayers/id/113.html",
    "http://www.plusliga.pl/statsPlayers/id/78.html",
    "http://www.plusliga.pl/statsPlayers/id/29821.html",
    "http://www.plusliga.pl/statsPlayers/id/2100363.html",
    "http://www.plusliga.pl/statsPlayers/id/380.html",
    "http://www.plusliga.pl/statsPlayers/id/22790.html",
    "http://www.plusliga.pl/statsPlayers/id/30489.html",
    "http://www.plusliga.pl/statsPlayers/id/29756.html",
    "http://www.plusliga.pl/statsPlayers/id/2100522.html",
    "http://www.plusliga.pl/statsPlayers/id/28000.html",
    "http://www.plusliga.pl/statsPlayers/id/26870.html",
    "http://www.plusliga.pl/statsPlayers/id/30496.html",
    "http://www.plusliga.pl/statsPlayers/id/26900.html",
    "http://www.plusliga.pl/statsPlayers/id/2100528.html",
    "http://www.plusliga.pl/statsPlayers/id/272.html",
    "http://www.plusliga.pl/statsPlayers/id/6.html",
    "http://www.plusliga.pl/statsPlayers/id/358.html",
    "http://www.plusliga.pl/statsPlayers/id/28378.html",
    "http://www.plusliga.pl/statsPlayers/id/22708.html",
    "http://www.plusliga.pl/statsPlayers/id/26823.html",
    "http://www.plusliga.pl/statsPlayers/id/573.html",
    "http://www.plusliga.pl/statsPlayers/id/2100359.html",
    "http://www.plusliga.pl/statsPlayers/id/29811.html",
    "http://www.plusliga.pl/statsPlayers/id/477.html",
    "http://www.plusliga.pl/statsPlayers/id/29719.html",
    "http://www.plusliga.pl/statsPlayers/id/103.html",
    "http://www.plusliga.pl/statsPlayers/id/707.html",
    "http://www.plusliga.pl/statsPlayers/id/30353.html",
    "http://www.plusliga.pl/statsPlayers/id/29886.html",
    "http://www.plusliga.pl/statsPlayers/id/561.html",
    "http://www.plusliga.pl/statsPlayers/id/88.html",
    "http://www.plusliga.pl/statsPlayers/id/2100513.html",
    "http://www.plusliga.pl/statsPlayers/id/58.html",
    "http://www.plusliga.pl/statsPlayers/id/2100355.html",
    "http://www.plusliga.pl/statsPlayers/id/28395.html",
    "http://www.plusliga.pl/statsPlayers/id/235.html",
    "http://www.plusliga.pl/statsPlayers/id/370.html",
    "http://www.plusliga.pl/statsPlayers/id/2100360.html",
    "http://www.plusliga.pl/statsPlayers/id/22640.html",
    "http://www.plusliga.pl/statsPlayers/id/2100525.html",
    "http://www.plusliga.pl/statsPlayers/id/177.html",
    "http://www.plusliga.pl/statsPlayers/id/516.html",
    "http://www.plusliga.pl/statsPlayers/id/22709.html",
    "http://www.plusliga.pl/statsPlayers/id/2100357.html",
    "http://www.plusliga.pl/statsPlayers/id/22573.html",
    "http://www.plusliga.pl/statsPlayers/id/22782.html",
    "http://www.plusliga.pl/statsPlayers/id/22799.html",
    "http://www.plusliga.pl/statsPlayers/id/87.html",
    "http://www.plusliga.pl/statsPlayers/id/28390.html",
    "http://www.plusliga.pl/statsPlayers/id/32.html",
    "http://www.plusliga.pl/statsPlayers/id/289.html",
    "http://www.plusliga.pl/statsPlayers/id/29662.html",
    "http://www.plusliga.pl/statsPlayers/id/22796.html",
    "http://www.plusliga.pl/statsPlayers/id/26880.html",
    "http://www.plusliga.pl/statsPlayers/id/430.html",
    "http://www.plusliga.pl/statsPlayers/id/22572.html",
    "http://www.plusliga.pl/statsPlayers/id/424.html",
    "http://www.plusliga.pl/statsPlayers/id/164.html",
    "http://www.plusliga.pl/statsPlayers/id/2100034.html",
    "http://www.plusliga.pl/statsPlayers/id/22526.html",
    "http://www.plusliga.pl/statsPlayers/id/45.html",
    "http://www.plusliga.pl/statsPlayers/id/153.html",
    "http://www.plusliga.pl/statsPlayers/id/107.html",
    "http://www.plusliga.pl/statsPlayers/id/570.html",
    "http://www.plusliga.pl/statsPlayers/id/30329.html",
    "http://www.plusliga.pl/statsPlayers/id/2100500.html",
    "http://www.plusliga.pl/statsPlayers/id/83.html",
    "http://www.plusliga.pl/statsPlayers/id/2100001.html",
    "http://www.plusliga.pl/statsPlayers/id/22804.html",
    "http://www.plusliga.pl/statsPlayers/id/414.html",
    "http://www.plusliga.pl/statsPlayers/id/71.html",
    "http://www.plusliga.pl/statsPlayers/id/467.html",
    "http://www.plusliga.pl/statsPlayers/id/709.html",
    "http://www.plusliga.pl/statsPlayers/id/22528.html",
    "http://www.plusliga.pl/statsPlayers/id/382.html",
    "http://www.plusliga.pl/statsPlayers/id/22569.html",
    "http://www.plusliga.pl/statsPlayers/id/2100502.html",
    "http://www.plusliga.pl/statsPlayers/id/2100521.html",
    "http://www.plusliga.pl/statsPlayers/id/670.html",
    "http://www.plusliga.pl/statsPlayers/id/30508.html",
    "http://www.plusliga.pl/statsPlayers/id/162.html",
    "http://www.plusliga.pl/statsPlayers/id/2100354.html",
    "http://www.plusliga.pl/statsPlayers/id/282.html",
    "http://www.plusliga.pl/statsPlayers/id/29747.html",
    "http://www.plusliga.pl/statsPlayers/id/2100249.html",
    "http://www.plusliga.pl/statsPlayers/id/589.html",
    "http://www.plusliga.pl/statsPlayers/id/30498.html",
    "http://www.plusliga.pl/statsPlayers/id/656.html",
    "http://www.plusliga.pl/statsPlayers/id/2100367.html",
    "http://www.plusliga.pl/statsPlayers/id/2100353.html",
    "http://www.plusliga.pl/statsPlayers/id/30342.html",
    "http://www.plusliga.pl/statsPlayers/id/450.html",
    "http://www.plusliga.pl/statsPlayers/id/483.html",
    "http://www.plusliga.pl/statsPlayers/id/298.html",
    "http://www.plusliga.pl/statsPlayers/id/30341.html",
    "http://www.plusliga.pl/statsPlayers/id/51.html",
    "http://www.plusliga.pl/statsPlayers/id/2100155.html",
    "http://www.plusliga.pl/statsPlayers/id/35.html",
    "http://www.plusliga.pl/statsPlayers/id/22560.html",
    "http://www.plusliga.pl/statsPlayers/id/2100518.html",
    "http://www.plusliga.pl/statsPlayers/id/29785.html",
    "http://www.plusliga.pl/statsPlayers/id/2100157.html",
    "http://www.plusliga.pl/statsPlayers/id/30482.html",
    "http://www.plusliga.pl/statsPlayers/id/2100115.html",
    "http://www.plusliga.pl/statsPlayers/id/30484.html",
    "http://www.plusliga.pl/statsPlayers/id/704.html",
    "http://www.plusliga.pl/statsPlayers/id/27949.html",
    "http://www.plusliga.pl/statsPlayers/id/2100366.html",
    "http://www.plusliga.pl/statsPlayers/id/30506.html",
    "http://www.plusliga.pl/statsPlayers/id/624.html",
    "http://www.plusliga.pl/statsPlayers/id/2100531.html",
    "http://www.plusliga.pl/statsPlayers/id/2100151.html",
    "http://www.plusliga.pl/statsPlayers/id/29833.html",
    "http://www.plusliga.pl/statsPlayers/id/2100193.html",
    "http://www.plusliga.pl/statsPlayers/id/327.html",
    "http://www.plusliga.pl/statsPlayers/id/456.html",
    "http://www.plusliga.pl/statsPlayers/id/29969.html",
    "http://www.plusliga.pl/statsPlayers/id/2100497.html",
    "http://www.plusliga.pl/statsPlayers/id/68.html",
    "http://www.plusliga.pl/statsPlayers/id/2100503.html",
    "http://www.plusliga.pl/statsPlayers/id/649.html",
    "http://www.plusliga.pl/statsPlayers/id/22758.html",
    "http://www.plusliga.pl/statsPlayers/id/30297.html",
    "http://www.plusliga.pl/statsPlayers/id/252.html",
    "http://www.plusliga.pl/statsPlayers/id/22558.html",
    "http://www.plusliga.pl/statsPlayers/id/413.html",
    "http://www.plusliga.pl/statsPlayers/id/30505.html",
    "http://www.plusliga.pl/statsPlayers/id/2100147.html",
    "http://www.plusliga.pl/statsPlayers/id/29753.html",
    "http://www.plusliga.pl/statsPlayers/id/28581.html",
    "http://www.plusliga.pl/statsPlayers/id/633.html",
    "http://www.plusliga.pl/statsPlayers/id/2100358.html",
    "http://www.plusliga.pl/statsPlayers/id/630.html",
    "http://www.plusliga.pl/statsPlayers/id/27987.html",
    "http://www.plusliga.pl/statsPlayers/id/2100362.html",
    "http://www.plusliga.pl/statsPlayers/id/28372.html",
    "http://www.plusliga.pl/statsPlayers/id/28371.html",
    "http://www.plusliga.pl/statsPlayers/id/2100369.html",
    "http://www.plusliga.pl/statsPlayers/id/28110.html",
    "http://www.plusliga.pl/statsPlayers/id/291.html",
    "http://www.plusliga.pl/statsPlayers/id/2100348.html",
    "http://www.plusliga.pl/statsPlayers/id/2100351.html",
    "http://www.plusliga.pl/statsPlayers/id/30493.html",
    "http://www.plusliga.pl/statsPlayers/id/667.html",
    "http://www.plusliga.pl/statsPlayers/id/26868.html",
    "http://www.plusliga.pl/statsPlayers/id/29606.html",
    "http://www.plusliga.pl/statsPlayers/id/156.html",
    "http://www.plusliga.pl/statsPlayers/id/444.html",
    "http://www.plusliga.pl/statsPlayers/id/29834.html",
    "http://www.plusliga.pl/statsPlayers/id/2100512.html",
    "http://www.plusliga.pl/statsPlayers/id/2100365.html",
    "http://www.plusliga.pl/statsPlayers/id/2100368.html",
    "http://www.plusliga.pl/statsPlayers/id/73.html",
    "http://www.plusliga.pl/statsPlayers/id/3.html",
    "http://www.plusliga.pl/statsPlayers/id/179.html",
    "http://www.plusliga.pl/statsPlayers/id/29752.html",
    "http://www.plusliga.pl/statsPlayers/id/28.html",
    "http://www.plusliga.pl/statsPlayers/id/112.html",
    "http://www.plusliga.pl/statsPlayers/id/80.html",
    "http://www.plusliga.pl/statsPlayers/id/22802.html",
    "http://www.plusliga.pl/statsPlayers/id/81.html",
    "http://www.plusliga.pl/statsPlayers/id/30500.html",
    "http://www.plusliga.pl/statsPlayers/id/22805.html",
    "http://www.plusliga.pl/statsPlayers/id/372.html",
    "http://www.plusliga.pl/statsPlayers/id/620.html",
    "http://www.plusliga.pl/statsPlayers/id/60.html",
    "http://www.plusliga.pl/statsPlayers/id/442.html"
]

In [84]:
urls = urls[0:225]

In [85]:
urls

['http://www.plusliga.pl/statsPlayers/id/152.html',
 'http://www.plusliga.pl/statsPlayers/id/30339.html',
 'http://www.plusliga.pl/statsPlayers/id/118.html',
 'http://www.plusliga.pl/statsPlayers/id/30172.html',
 'http://www.plusliga.pl/statsPlayers/id/415.html',
 'http://www.plusliga.pl/statsPlayers/id/2100505.html',
 'http://www.plusliga.pl/statsPlayers/id/82.html',
 'http://www.plusliga.pl/statsPlayers/id/22767.html',
 'http://www.plusliga.pl/statsPlayers/id/277.html',
 'http://www.plusliga.pl/statsPlayers/id/631.html',
 'http://www.plusliga.pl/statsPlayers/id/297.html',
 'http://www.plusliga.pl/statsPlayers/id/293.html',
 'http://www.plusliga.pl/statsPlayers/id/30333.html',
 'http://www.plusliga.pl/statsPlayers/id/231.html',
 'http://www.plusliga.pl/statsPlayers/id/63.html',
 'http://www.plusliga.pl/statsPlayers/id/22550.html',
 'http://www.plusliga.pl/statsPlayers/id/101.html',
 'http://www.plusliga.pl/statsPlayers/id/488.html',
 'http://www.plusliga.pl/statsPlayers/id/2100506.htm

In [86]:
df_all = pd.DataFrame()
for url in urls:
    df=scrap_table_to_df_and_overall_stats(url)
    df_all = df_all.append(df)
df_all.shape

(3158, 25)

In [87]:
df_all.shape

(3158, 25)

In [88]:
df_all['Gracz'].value_counts()

Przemysław Smoliński        22
Bartłomiej Lipiński         22
Miguel Tavares Rodrigues    22
Jędrzej Gruszczyński        22
Jakub Ziobrowski            22
Artur Ratajczak             21
Janusz Gałązka              21
Tomasz Fornal               21
Dawid Konarski              21
Jan Firlej                  21
Rafał Faryna                21
Luke Perry                  21
Andreas Takvam              21
Antoine Brizard             21
Bartłomiej Bołądź           21
Arpad Baróti                21
Kamil Semeniuk              21
Michał Masny                21
Dawid Woch                  21
Dustin Watten               21
Piotr Lipiński              21
Grzegorz Łomacz             21
Lukas Kampa                 21
Adrian Stańczak             21
Damian Wojtaszek            21
Kawika Shoji                21
Mateusz Malinowski          21
Nicolas Marechal            21
David Sossenheimer          21
Michał Potera               21
                            ..
Łukasz Kaczmarek             8
Dawid So

In [95]:
df_all = df_all.reset_index()

In [96]:
df_all.to_csv('pls_stats.csv')

In [97]:
df_all

,index,Rozegrane mecze,Sety,Punkty,Zagrywka_Suma,Zagrywka_As,Zagrywka_Błąd,Zagrywka_Asy na set,Przyjęcie_Suma,Przyjęcie_Błąd,...,Atak_Perf,Atak_% perf,Blok_Pkt,Blok_Pkt na set,Gracz,Liczba sezonów w lidze,Liczba meczy w lidze,Liczba pkt w lidze,Liczba asów w lidze,Liczba bloków w lidze
0,0,Asseco Resovia Rzeszów -BKS Visła Bydgoszcz,3.0,1.0,4.0,1.0,0.0,"0,33",0.0,0.0,...,0.0,"0,00",0.0,"0,00",Alek Achrem,9,166,2007,152,181
1,1,Asseco Resovia Rzeszów -MKS Będzin,4.0,1.0,8.0,1.0,3.0,"0,25",0.0,0.0,...,0.0,"0,00",0.0,"0,00",Alek Achrem,9,166,2007,152,181
2,2,Asseco Resovia Rzeszów -Grupa Azoty ZAKSA Kędz...,2.0,0.0,2.0,0.0,2.0,"0,00",0.0,0.0,...,0.0,"0,00",0.0,"0,00",Alek Achrem,9,166,2007,152,181
3,3,Asseco Resovia Rzeszów -Cerrad Enea Czarni Radom,3.0,4.0,6.0,0.0,2.0,"0,00",13.0,2.0,...,4.0,"100,00",0.0,"0,00",Alek Achrem,9,166,2007,152,181
4,0,Indykpol AZS Olsztyn -Cerrad Enea Czarni Radom,5.0,5.0,16.0,0.0,4.0,"0,00",23.0,0.0,...,4.0,"30,77",1.0,"0,20",Robbert Andringa,3,72,642,34,97
5,1,Indykpol AZS Olsztyn -Aluron Virtu CMC Zawiercie,4.0,10.0,12.0,0.0,3.0,"0,00",26.0,3.0,...,10.0,"55,56",0.0,"0,00",Robbert Andringa,3,72,642,34,97
6,2,Indykpol AZS Olsztyn -Asseco Resovia Rzeszów,4.0,7.0,11.0,0.0,2.0,"0,00",14.0,0.0,...,4.0,"26,67",3.0,"0,75",Robbert Andringa,3,72,642,34,97
7,3,Indykpol AZS Olsztyn -GKS Katowice,5.0,12.0,19.0,2.0,4.0,"0,40",21.0,1.0,...,9.0,"39,13",1.0,"0,20",Robbert Andringa,3,72,642,34,97
8,4,Indykpol AZS Olsztyn -Trefl Gdańsk,2.0,1.0,6.0,0.0,2.0,"0,00",9.0,0.0,...,1.0,"10,00",0.0,"0,00",Robbert Andringa,3,72,642,34,97
9,5,Indykpol AZS Olsztyn -Ślepsk Malow Suwałki,2.0,7.0,5.0,0.0,2.0,"0,00",12.0,0.0,...,5.0,"45,45",2.0,"1,00",Robbert Andringa,3,72,642,34,97
